<a href="https://colab.research.google.com/github/SonalGire/RAG-based-RFI-Projects/blob/main/RAG_quizz_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 **IT Literacy Quiz Generator (PDF to Objective Questions) :**


This project auto-generates True/False, Yes/No, and MCQ questions from any IT Literacy PDF (like SOPs or manuals) using Gemini 1.5 Flash. It extracts content, generates questions via LLM, groups them by type, and exports a clean, printable PDF quiz with answers on the last page.


✅ Why? To save time and effort in creating quizzes manually.

👩‍🏫 Use it for: IT trainers, educators, students, HR/compliance teams.

📄 Just upload PDF → Get ready-to-use quiz in minutes.



In [ ]:
!pip install PyPDF2 fpdf google-generativeai python-dotenv


In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = input("🔑 Enter your Gemini API key: ")


In [29]:
import os
import re
import traceback
from PyPDF2 import PdfReader
from PyPDF2.errors import PdfReadError
from fpdf import FPDF
import google.generativeai as genai

# ---------------- GOOGLE API KEY ----------------

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])



# ---------------- CLEAN UNICODE FOR PDF ----------------
def clean_unicode(text):
    replacements = {
        "\u201c": '"', "\u201d": '"',  # smart double quotes
        "\u2018": "'", "\u2019": "'",  # smart single quotes
        "\u2013": "-", "\u2014": "-",  # en/em dashes
        "\u2022": "*",                # bullet
        "\xa0": " ",                  # non-breaking space
    }
    for bad, good in replacements.items():
        text = text.replace(bad, good)
    return text.encode("latin-1", "ignore").decode("latin-1")  # strip anything left

# ---------------- PDF TEXT LOADER ----------------
def load_pdf_text(path: str, max_chars: int = None) -> str:
    try:
        reader = PdfReader(path)
        full_text = "\n".join([page.extract_text() or "" for page in reader.pages])
        return full_text[:max_chars] if max_chars else full_text
    except Exception as e:
        print(f"❌ ERROR reading PDF: {e}")
        return ""

pdf_path = "/content/IT_computer_literacy.pdf"     #🔁 Replace with your actual file path
pdf_text = load_pdf_text(pdf_path, max_chars=5000)





In [30]:
# ========================================================
# Prompt for IT Literacy PDF-Based Question Generation
# ========================================================

def build_prompt(document_text):
    return f"""
You are an IT Literacy Instructor and expert in computer education.

From the instructional manual below, generate **30 objective questions** that assess understanding of basic IT concepts such as computer hardware, software, internet, file types, networking, and security practices.

Avoid factual or trivial questions like:
❌ What is written on page X
❌ Dates, model numbers, or versions
❌ Names of software products or companies
❌ Questions that just test rote memory

✅ Instead, focus on meaningful conceptual and applied questions, such as:
- What is the role of an operating system in a computer?
- Why is it important to use strong passwords?
- What is the difference between RAM and storage?
- When should antivirus software be updated?


Generate:
1. 10 True/False questions (Q1–Q10)
2. 10 Yes/No questions (Q11–Q20)
3. 10 MCQs (Q21–Q30) – these should test concepts, reasoning, or real-world application

📌 Format every question like this (even T/F and Yes/No):

Q<number>. <Question>
A. Option A
B. Option B
C. Option C
D. Option D
Answer: <Correct option letter>

Use only the content provided in the manual to form all questions and options.

Manual:
{document_text}
"""


In [34]:
# ---------------- QUESTION PARSER ----------------
def parse_questions(text):
    tf, yn, mcq = [], [], []
    tf_ans, yn_ans, mcq_ans = [], [], []
    lines = text.strip().splitlines()
    current = None
    qnum = 0
    for line in lines:
        if re.match(r"^Q([1-9]|10)\.", line):
            current, qnum = "tf", int(re.findall(r"\d+", line)[0])
            tf.append(line)
        elif re.match(r"^Q(1[1-9]|20)\.", line):
            current, qnum = "yn", int(re.findall(r"\d+", line)[0])
            yn.append(line)
        elif re.match(r"^Q(2[1-9]|30)\.", line):
            current, qnum = "mcq", int(re.findall(r"\d+", line)[0])
            mcq.append(line)
        elif line.startswith(("A.", "B.", "C.", "D.")):
            (tf if current == "tf" else yn if current == "yn" else mcq).append(line)
        elif line.startswith("Answer:"):
            ans = f"{qnum}. {line.split(':')[1].strip()}"
            if current == "tf": tf_ans.append(ans)
            elif current == "yn": yn_ans.append(ans)
            else: mcq_ans.append(ans)
    return tf, yn, mcq, tf_ans, yn_ans, mcq_ans

In [35]:
# ---------------- PDF FORMATTER ----------------
class StyledPDF(FPDF):
    def header(self):
        pass
    def chapter_title(self, title):
        self.set_fill_color(200, 220, 255)
        self.set_text_color(0, 0, 80)
        self.set_font("Arial", "B", 15)
        self.cell(0, 12, title, ln=True, fill=True)
        self.ln(6)

    def question_block(self, lines):
        self.set_font("Arial", size=12)
        self.set_text_color(20, 20, 20)
        for line in lines:
            self.multi_cell(0, 8, line)
        self.ln(3)

    def answer_block(self, lines):
        self.set_fill_color(245, 245, 245)
        self.set_text_color(60, 60, 60)
        self.set_font("Arial", "", 12)
        for line in lines:
            self.cell(0, 8, line, ln=True, fill=True)
        self.ln(4)



# ---------------- EXPORT PDF ----------------
def export_to_pdf(tf, yn, mcq, tf_ans, yn_ans, mcq_ans, output_file="IT_Literacy_Quiz.pdf"):
    pdf = StyledPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    # Cover Page
    pdf.add_page()
    pdf.set_font("Arial", "B", 22)
    pdf.set_text_color(0, 50, 100)
    pdf.cell(0, 70, "", ln=True)  # vertical spacing
    pdf.cell(0, 10, "IT Literacy Objective Quiz", ln=True, align="C")
    pdf.ln(10)
    pdf.set_font("Arial", "", 14)
    pdf.set_text_color(80, 80, 80)
    pdf.multi_cell(0, 8, "This document contains beginner to intermediate-level objective questions on IT and Computer Literacy. The questions are generated from public training manuals to enhance digital awareness and practical understanding.")

    def write_section(title, questions):
        pdf.chapter_title(title)
        block = []
        for line in questions:
            if line.startswith("Q") and block:
                pdf.question_block(block)
                block = []
            block.append(line)
        if block:
            pdf.question_block(block)

    pdf.add_page()
    write_section("Section 1: True/False Questions :", tf)

    pdf.add_page()
    write_section("Section 2: Yes/No Questions :", yn)

    pdf.add_page()
    write_section("Section 3: Multiple Choice Questions :", mcq)

    # Answer Page
    pdf.add_page()
    pdf.chapter_title("Answer Key :")
    pdf.set_font("Arial", "B", 13)
    pdf.set_text_color(0, 0, 100)
    pdf.cell(0, 10, "True/False Answers :", ln=True)
    pdf.answer_block(tf_ans)

    pdf.cell(0, 10, "Yes/No Answers :", ln=True)
    pdf.answer_block(yn_ans)

    pdf.cell(0, 10, "MCQ Answers", ln=True)
    pdf.answer_block(mcq_ans)

    pdf.output(output_file)
    print(f"✅ Styled PDF exported: {output_file}")


    # ---------------- MAIN FLOW ----------------
if __name__ == "__main__":
    path = "/content/IT_computer_literacy.pdf"
    text = load_pdf_text(path)

    if not text:
        print("❌ SOP text extraction failed.")
        exit(1)

    try:
        model = genai.GenerativeModel("gemini-1.5-flash")
        prompt = build_prompt(text)

        print("🔄 Generating questions...")
        response = model.generate_content(prompt)
        question_text = clean_unicode(response.text)

        tf, yn, mcq, tf_ans, yn_ans, mcq_ans = parse_questions(question_text)
        export_to_pdf(tf, yn, mcq, tf_ans, yn_ans, mcq_ans)

    except Exception as e:
        print("❌ Error during Gemini processing:")
        traceback.print_exc()


🔄 Generating questions...
✅ Styled PDF exported: IT_Literacy_Quiz.pdf
